In [1]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

df_encoded=pd.read_csv('Final_Anime_Dataset.csv')
user_data_temp_final_path = "user_data_temp_final.csv"
OneUser_multiAnime_df = pd.read_csv(user_data_temp_final_path)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_19040\3360529992.py:7: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_encoded=pd.read_csv('Final_Anime_Dataset.csv')


In [2]:
OneUser_multiAnime_df.columns

Index(['Username', 'Gender', 'Mean Score', 'Completed', 'Birth_Year',
       'user_id', 'anime_id', 'Anime Title', 'rating'],
      dtype='object')

In [3]:
if 'Unnamed: 0' in OneUser_multiAnime_df.columns:
    OneUser_multiAnime_df.drop(columns=['Unnamed: 0'], inplace=True)

In [4]:
df_encoded

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Episodes,Aired,Premiered,...,"Studio_pH Studio, D & D Pictures","Studio_pH Studio, Noovo",Studio_production doA,Studio_studio MOTHER,Studio_studio YOG,Studio_trenova,Studio_ufotable,"Studio_ufotable, Shaft, A-1 Pictures, SILVER LINK., Lerche, Lay-duce, CloverWorks, Drive","Studio_ufotable, feel., Studio Flag",Release_Year
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",26,"Apr 3, 1998 to Apr 24, 1999",spring 1998,...,0,0,0,0,0,0,0,0,0,1998.0
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",1,"Sep 1, 2001",UNKNOWN,...,0,0,0,0,0,0,0,0,0,2001.0
2,6,Trigun,Trigun,トライガン,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",26,"Apr 1, 1998 to Sep 30, 1998",spring 1998,...,0,0,0,0,0,0,0,0,0,1998.0
3,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,26,"Jul 3, 2002 to Dec 25, 2002",summer 2002,...,0,0,0,0,0,0,0,0,0,2002.0
4,8,Bouken Ou Beet,Beet the Vandel Buster,冒険王ビィト,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,52,"Sep 30, 2004 to Sep 29, 2005",fall 2004,...,0,0,0,0,0,0,0,0,0,2004.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19843,55731,Wu Nao Monu,UNKNOWN,无脑魔女,NaN,"Comedy, Fantasy, Slice of Life",No description available for this anime.,15,"Jul 4, 2023 to ?",UNKNOWN,...,0,0,0,0,0,0,0,0,0,2023.0
19844,55732,Bu Xing Si: Yuan Qi,Blader Soul,捕星司·源起,NaN,"Action, Adventure, Fantasy",No description available for this anime.,18,"Jul 27, 2023 to ?",UNKNOWN,...,0,0,0,0,0,0,0,0,0,2023.0
19845,55733,Di Yi Xulie,The First Order,第一序列,NaN,"Action, Adventure, Fantasy, Sci-Fi",No description available for this anime.,16,"Jul 19, 2023 to ?",UNKNOWN,...,0,0,0,0,0,0,0,0,0,2023.0
19846,55734,Bokura no Saishuu Sensou,UNKNOWN,僕らの最終戦争,NaN,UNKNOWN,A music video for the song Bokura no Saishuu S...,1,"Apr 23, 2022",UNKNOWN,...,0,0,0,0,0,0,0,0,0,2022.0


In [22]:


class DataFrameMergerTransformer(BaseEstimator, TransformerMixin):
    """
    A scikit-learn transformer that merges a DataFrame with another reference DataFrame.
    
    Parameters:
    -----------
    df_encoded : pandas.DataFrame
        The main DataFrame containing all columns
    on : str or list of str
        Column(s) to join on
    how : str, default='inner'
        Type of merge to be performed ('inner', 'left', 'right', 'outer')
    suffixes : tuple of str, default=('_x', '_y')
        Suffixes to use for overlapping columns
    reset_index : bool, default=True
        Whether to reset the index of the resulting merged DataFrame
    drop_on_duplicate : bool or list, default=False
        Whether to drop duplicate columns after merging (except the join key)
        If list, specifies the columns to drop
    """
    
    def __init__(self, df_encoded, on, how='inner', suffixes=('_x', '_y'), 
                 reset_index=True, drop_on_duplicate=False):
        self.df_encoded = df_encoded
        self.on = on
        self.how = how
        self.suffixes = suffixes
        self.reset_index = reset_index
        self.drop_on_duplicate = drop_on_duplicate
    
    def fit(self, X, y=None):
        """Fit method (no actual fitting needed)"""
        return self
    
    def transform(self, X):
        """
        Merge the input DataFrame with the reference DataFrame.
        
        Parameters:
        -----------
        X : pandas.DataFrame
            DataFrame to be merged with the reference DataFrame
            
        Returns:
        --------
        pandas.DataFrame
            Merged DataFrame
        """
        # Extract the required columns to create df_anime_genre
        df_anime_genre = self.df_encoded[[
            'anime_id', 'Genres', 'Genre_Action', 'Genre_Adventure', 'Genre_Avant Garde',
            'Genre_Award Winning', 'Genre_Boys Love', 'Genre_Comedy', 'Genre_Drama',
            'Genre_Ecchi', 'Genre_Erotica', 'Genre_Fantasy', 'Genre_Girls Love',
            'Genre_Gourmet', 'Genre_Hentai', 'Genre_Horror', 'Genre_Mystery',
            'Genre_Romance', 'Genre_Sci-Fi', 'Genre_Slice of Life', 'Genre_Sports',
            'Genre_Supernatural', 'Genre_Suspense'
        ]]
        
        # Perform the merge operation
        merged_df = pd.merge(
            X, 
            df_anime_genre,
            on=self.on,
            how=self.how,
            suffixes=self.suffixes
        )
        
        # Reset index if specified
        if self.reset_index:
            merged_df = merged_df.reset_index(drop=True)
        
        # Handle duplicate columns
        if self.drop_on_duplicate:
            if isinstance(self.drop_on_duplicate, list):
                # Drop specific columns
                merged_df = merged_df.drop(columns=self.drop_on_duplicate, errors='ignore')
            else:
                # Identify and drop columns with '_y' suffix (from reference_df)
                duplicate_cols = [col for col in merged_df.columns if col.endswith(self.suffixes[1])]
                merged_df = merged_df.drop(columns=duplicate_cols, errors='ignore')
        
        return merged_df

class UserGenreRatingAggregator(BaseEstimator, TransformerMixin):
    """
    A transformer that aggregates user ratings for different genres and creates a user profile
    with average ratings for each genre.
    
    Parameters:
    -----------
    genre_prefix : str, default='Genre_'
        Prefix to identify genre columns
    rating_col : str, default='rating'
        Column name containing ratings
    user_id_col : str, default='user_id'
        Column name containing user IDs
    user_info_cols : list or None, default=None
        Additional user columns to include in the result
    output_prefix : str, default='avg_'
        Prefix for the output average columns
    round_decimals : int or None, default=2
        Number of decimals to round to (None for no rounding)
    """
    
    def __init__(self, genre_prefix='Genre_', rating_col='rating', user_id_col='user_id',
                 user_info_cols=None, output_prefix='avg_', round_decimals=2):
        self.genre_prefix = genre_prefix
        self.rating_col = rating_col
        self.user_id_col = user_id_col
        self.user_info_cols = user_info_cols if user_info_cols is not None else [
            'Username', 'Gender', 'Birth_Year', 'Mean Score', 'Completed'
        ]
        self.output_prefix = output_prefix
        self.round_decimals = round_decimals
    
    def fit(self, X, y=None):
        """Fit method (no actual fitting needed)"""
        return self
    
    def transform(self, X):
        """
        Transform the input DataFrame to create user profiles with average genre ratings.
        
        Parameters:
        -----------
        X : pandas.DataFrame
            Input DataFrame with user ratings and genre information
            
        Returns:
        --------
        pandas.DataFrame
            DataFrame with one row per user and average ratings for each genre
        """
        # Make a copy to avoid modifying the original
        X_copy = X.copy()
        
        # Identify genre columns
        genre_cols = [col for col in X_copy.columns if col.startswith(self.genre_prefix)]
        
        # Calculate weighted ratings (genre * rating)
        rating_matrix = X_copy[genre_cols].multiply(X_copy[self.rating_col], axis=0)
        
        # Group by user_id and calculate sums
        rating_sums = rating_matrix.groupby(X_copy[self.user_id_col]).sum()
        genre_counts = X_copy.groupby(self.user_id_col)[genre_cols].sum()
        
        # Compute average rating per genre
        avg_genre_ratings = rating_sums.div(genre_counts)
        
        # Rename columns to use output prefix
        avg_genre_ratings.columns = [
            col.replace(self.genre_prefix, self.output_prefix) for col in avg_genre_ratings.columns
        ]
        
        # Get user info
        user_cols = [self.user_id_col] + [col for col in self.user_info_cols if col in X_copy.columns]
        user_info = X_copy[user_cols].drop_duplicates(self.user_id_col).set_index(self.user_id_col)
        
        # Combine results
        result_df = pd.concat([user_info, avg_genre_ratings], axis=1).reset_index()
        
        # Round if specified
        if self.round_decimals is not None:
            avg_cols = avg_genre_ratings.columns
            result_df[avg_cols] = result_df[avg_cols].round(self.round_decimals)
        
        return result_df

class GenreRatingTypeConverter(BaseEstimator, TransformerMixin):
    """
    A transformer that converts all columns with a specific prefix to numeric data types,
    handling non-numeric values by converting them to NaN.
    
    Parameters:
    -----------
    column_prefix : str, default='avg_'
        Prefix used to identify columns for conversion
    errors : str, default='coerce'
        How to handle errors in conversion:
        - 'ignore': leave invalid values as is
        - 'raise': raise an exception
        - 'coerce': convert invalid values to NaN
    downcast : str or None, default=None
        Type to downcast to if possible ('integer', 'signed', 'unsigned', 'float')
    """
    
    def __init__(self, column_prefix='avg_', errors='coerce', downcast=None):
        self.column_prefix = column_prefix
        self.errors = errors
        self.downcast = downcast
        self.columns_converted_ = None
    
    def fit(self, X, y=None):
        """
        Identify columns to convert based on prefix.
        
        Parameters:
        -----------
        X : pandas.DataFrame
            Input DataFrame
        y : array-like, default=None
            Not used, present for API consistency
            
        Returns:
        --------
        self
        """
        # Identify columns starting with the specified prefix
        self.columns_converted_ = [col for col in X.columns if col.startswith(self.column_prefix)]
        return self
    
    def transform(self, X):
        """
        Convert identified columns to numeric types.
        
        Parameters:
        -----------
        X : pandas.DataFrame
            Input DataFrame to transform
            
        Returns:
        --------
        pandas.DataFrame
            Transformed DataFrame with numeric columns
        """
        X_result = X.copy()
        
        for column in self.columns_converted_:
            X_result[column] = pd.to_numeric(X_result[column], 
                                         errors=self.errors, 
                                         downcast=self.downcast)
        
        return X_result
    
    def get_feature_names_out(self, input_features=None):
        """
        Get output feature names.
        
        Parameters:
        -----------
        input_features : array-like of str or None, default=None
            Input features
            
        Returns:
        --------
        list
            List of converted column names
        """
        return self.columns_converted_

class GenderOneHotEncoder(BaseEstimator, TransformerMixin):
    """
    A transformer that one-hot encodes the 'Gender' column in a DataFrame.
    
    Parameters:
    -----------
    column_name : str, default='Gender'
        Name of the column to one-hot encode
    drop : str or None, default=None
        Drop strategy for encoder ('first', 'if_binary', or None)
    sparse_output : bool, default=False
        Whether to return a sparse matrix
    """
    
    def __init__(self, column_name='Gender', drop=None, sparse_output=False):
        self.column_name = column_name
        self.drop = drop
        self.sparse_output = sparse_output
        self.encoder = None
        self.feature_names_out_ = None
    
    def fit(self, X, y=None):
        """
        Fit the one-hot encoder on the specified column.
        
        Parameters:
        -----------
        X : pandas.DataFrame
            Input DataFrame
        y : array-like, default=None
            Not used, present for API consistency
            
        Returns:
        --------
        self
        """
        # Initialize the encoder
        self.encoder = OneHotEncoder(sparse_output=self.sparse_output, drop=self.drop, dtype=int)
        
        # Fit the encoder if the column exists
        if self.column_name in X.columns:
            self.encoder.fit(X[[self.column_name]])
            self.feature_names_out_ = self.encoder.get_feature_names_out([self.column_name])
        else:
            self.feature_names_out_ = []
        
        return self
    
    def transform(self, X):
        """
        Transform the DataFrame by one-hot encoding the specified column.
        
        Parameters:
        -----------
        X : pandas.DataFrame
            Input DataFrame
            
        Returns:
        --------
        pandas.DataFrame
            Transformed DataFrame with one-hot encoded column
        """
        X_result = X.copy()
        
        # Only transform if the column exists
        if self.column_name in X_result.columns:
            # Encode the column
            encoded_array = self.encoder.transform(X_result[[self.column_name]])
            
            # Convert to DataFrame with proper column names
            encoded_df = pd.DataFrame(
                encoded_array, 
                columns=self.feature_names_out_,
                index=X_result.index
            )
            
            # Concatenate with the original DataFrame
            X_result = pd.concat([X_result, encoded_df], axis=1)
            
            # Drop the original column
            X_result.drop(columns=[self.column_name], inplace=True)
        
        return X_result
    
    def get_feature_names_out(self, input_features=None):
        """
        Get output feature names.
        
        Parameters:
        -----------
        input_features : array-like of str or None, default=None
            Input features
            
        Returns:
        --------
        list
            List of feature names after transformation
        """
        return self.feature_names_out_

class UserAgeProcessor(BaseEstimator, TransformerMixin):
    """
    A transformer that processes age-related information in the user data:
    1. Calculates current age from Birth_Year
    2. Creates age group categories
    
    Parameters:
    -----------
    birth_year_col : str, default='Birth_Year'
        Column containing birth year
    current_year : int, default=2025
        Current year for age calculation
    add_age_groups : bool, default=True
        Whether to add age group categorization
    """
    
    def __init__(self, birth_year_col='Birth_Year', current_year=2025, add_age_groups=True):
        self.birth_year_col = birth_year_col
        self.current_year = current_year
        self.add_age_groups = add_age_groups
        self.age_col = 'Age'
        self.age_group_cols = [
            'Age_Group__Gen_Alpha', 'Age_Group__Zoomers',
            'Age_Group__Millennials', 'Age_Group__Gen_X',
            'Age_Group__Boomers_Plus'
        ]
    
    def fit(self, X, y=None):
        """Fit method (no actual fitting needed)"""
        return self
    
    def transform(self, X):
        """
        Transform the DataFrame by adding age and age group columns.
        
        Parameters:
        -----------
        X : pandas.DataFrame
            Input DataFrame with Birth_Year column
            
        Returns:
        --------
        pandas.DataFrame
            Transformed DataFrame with additional age columns
        """
        X_result = X.copy()
        
        # Calculate age if birth year column exists
        if self.birth_year_col in X_result.columns:
            X_result[self.age_col] = self.current_year - X_result[self.birth_year_col]
            
            # Add age groups if specified
            if self.add_age_groups:
                # Define age groups
                X_result['Age_Group__Gen_Alpha'] = (X_result[self.age_col] < 13).astype(int)
                X_result['Age_Group__Zoomers'] = ((X_result[self.age_col] >= 13) & 
                                                 (X_result[self.age_col] < 28)).astype(int)
                X_result['Age_Group__Millennials'] = ((X_result[self.age_col] >= 28) & 
                                                     (X_result[self.age_col] < 43)).astype(int)
                X_result['Age_Group__Gen_X'] = ((X_result[self.age_col] >= 43) & 
                                               (X_result[self.age_col] < 59)).astype(int)
                X_result['Age_Group__Boomers_Plus'] = (X_result[self.age_col] >= 59).astype(int)
            
            # Drop birth year if it was used
            X_result.drop(columns=[self.birth_year_col], inplace=True)
        
        return X_result
    
    def get_feature_names_out(self, input_features=None):
        """Get output feature names."""
        feature_names = [self.age_col]
        if self.add_age_groups:
            feature_names.extend(self.age_group_cols)
        return feature_names

class UserFeatureSelector(BaseEstimator, TransformerMixin):
    """
    A transformer that selects and reorders user features for the X_U matrix.
    
    Parameters:
    -----------
    features : list or None, default=None
        List of feature columns to select. If None, uses a default set.
    """
    
    def __init__(self, features=None):
        if features is None:
            self.features = [
                'Mean Score', 'Completed',
                'avg_Action', 'avg_Adventure', 'avg_Avant Garde', 'avg_Award Winning',
                'avg_Boys Love', 'avg_Comedy', 'avg_Drama', 'avg_Ecchi', 'avg_Erotica',
                'avg_Fantasy', 'avg_Girls Love', 'avg_Gourmet', 'avg_Hentai',
                'avg_Horror', 'avg_Mystery', 'avg_Romance', 'avg_Sci-Fi',
                'avg_Slice of Life', 'avg_Sports', 'avg_Supernatural', 'avg_Suspense',
                'Gender_Female', 'Gender_Male', 'Gender_Non-Binary','Age',
                 'Age_Group__Gen_Alpha', 'Age_Group__Zoomers',
                'Age_Group__Millennials', 'Age_Group__Gen_X', 'Age_Group__Boomers_Plus'
            ]
        else:
            self.features = features
    
    def fit(self, X, y=None):
        """Fit method (no actual fitting needed)"""
        return self
    
    def transform(self, X):
        """
        Select and reorder features.
        
        Parameters:
        -----------
        X : pandas.DataFrame
            Input DataFrame
            
        Returns:
        --------
        pandas.DataFrame
            DataFrame with selected features
        """
        # Get only columns that exist in the input
        available_features = [col for col in self.features if col in X.columns]
        
        # Return selected columns
        return X[available_features]
    
    def get_feature_names_out(self, input_features=None):
        """Get output feature names."""
        return self.features

class MissingValueImputer(BaseEstimator, TransformerMixin):
    """
    A transformer that imputes missing values in the user feature matrix.
    
    Parameters:
    -----------
    strategy : str, default='mean'
        Imputation strategy ('mean', 'median', 'most_frequent', 'constant')
    fill_value : any, default=None
        Used when strategy is 'constant'
    """
    
    def __init__(self, strategy='mean', fill_value=None):
        self.strategy = strategy
        self.fill_value = fill_value
        self.imputer = SimpleImputer(strategy=strategy, fill_value=fill_value)
        self.feature_names_in_ = None
    
    def fit(self, X, y=None):
        """
        Fit the imputer to the data.
        
        Parameters:
        -----------
        X : pandas.DataFrame
            Input DataFrame
        y : array-like, default=None
            Not used, present for API consistency
            
        Returns:
        --------
        self
        """
        self.feature_names_in_ = X.columns
        self.imputer.fit(X)
        return self
    
    def transform(self, X):
        """
        Impute missing values in the data.
        
        Parameters:
        -----------
        X : pandas.DataFrame
            Input DataFrame
            
        Returns:
        --------
        pandas.DataFrame
            DataFrame with imputed values
        """
        imputed_data = self.imputer.transform(X)
        return pd.DataFrame(imputed_data, columns=self.feature_names_in_, index=X.index)
    
    def get_feature_names_out(self, input_features=None):
        """Get output feature names."""
        return list(self.feature_names_in_)

class FeaturesStandardizer(BaseEstimator, TransformerMixin):
    """
    A transformer that standardizes features using StandardScaler.
    
    Parameters:
    -----------
    with_mean : bool, default=True
        Whether to center the data before scaling
    with_std : bool, default=True
        Whether to scale the data to unit variance
    """
    
    def __init__(self, with_mean=True, with_std=True):
        self.with_mean = with_mean
        self.with_std = with_std
        self.scaler = StandardScaler(with_mean=with_mean, with_std=with_std)
        self.feature_names_in_ = None
    
    def fit(self, X, y=None):
        """
        Fit the scaler to the data.
        
        Parameters:
        -----------
        X : pandas.DataFrame
            Input DataFrame
        y : array-like, default=None
            Not used, present for API consistency
            
        Returns:
        --------
        self
        """
        self.feature_names_in_ = X.columns
        self.scaler.fit(X)
        return self
    
    def transform(self, X):
        """
        Standardize the data.
        
        Parameters:
        -----------
        X : pandas.DataFrame
            Input DataFrame
            
        Returns:
        --------
        pandas.DataFrame
            DataFrame with standardized values
        """
        scaled_data = self.scaler.transform(X)
        return pd.DataFrame(scaled_data, columns=self.feature_names_in_, index=X.index)
    
    def get_feature_names_out(self, input_features=None):
        """Get output feature names."""
        return list(self.feature_names_in_)

## ====================== CORRECTED IMPUTER ====================== ##
class BaseDataImputer(MissingValueImputer):
    """
    Modified imputer that uses base dataset statistics for transformation
    """
    def __init__(self, base_data, strategy='mean', fill_value=None):
        super().__init__(strategy, fill_value)
        self.base_data = base_data
        
    def fit(self, X=None, y=None):
        """Fit on base data instead of input X"""
        self.imputer.fit(self.base_data)
        self.feature_names_in_ = self.base_data.columns
        return self

## ====================== UPDATED PIPELINE ====================== ##

def create_user_preprocessing_pipeline(df_encoded, user_ratings_df=None, base_data=None):
    """Modified pipeline with base data support"""
    # Stage 1: Base data preprocessing
    base_preprocessor = Pipeline([
        ('merge_genres', DataFrameMergerTransformer(df_encoded, on='anime_id')),
        ('aggregate_ratings', UserGenreRatingAggregator()),
        ('convert_genre_types', GenreRatingTypeConverter()),
        ('encode_gender', GenderOneHotEncoder()),
        ('process_age', UserAgeProcessor()),
        ('select_features', UserFeatureSelector())
    ])
    
    # Process base data if provided
    if base_data is not None:
        base_preprocessed = base_preprocessor.fit_transform(base_data)
    else:
        base_preprocessed = None
        
    # Stage 2: Main processing pipeline
    pipeline = Pipeline([
        ('preprocessor', base_preprocessor),
        ('impute_missing', BaseDataImputer(base_data=base_preprocessed) 
            if base_preprocessed is not None 
            else MissingValueImputer()),
        ('standardize', FeaturesStandardizer())
    ])
    
    return pipeline
## ====================== CORRECTED USAGE EXAMPLE ====================== ##
if __name__ == "__main__":
    # Load base data
    base_df = pd.read_csv("user_data_temp_final.csv")
    df_encoded = pd.read_csv("Final_Anime_Dataset.csv")
    
    # 1. Create and FIT the pipeline on base data
    base_pipeline = create_user_preprocessing_pipeline(
        df_encoded=df_encoded,
        base_data=base_df  # This tells the pipeline to use base data stats
    )
    
    # Explicitly fit the pipeline (critical step!)
    base_pipeline.fit(base_df)  # Now all components are properly fitted
    
    # 2. Process new users using fitted components
    new_user_raw = pd.read_csv("user_8974_ratings_20250414_224401.csv")
    
    try:
        processed_new = base_pipeline.transform(new_user_raw)
    except ValueError as e:
        print(f"Error: {str(e)}")
        print("Common fixes:")
        print("- Ensure new data has all required columns from base dataset")
        print("- Verify anime IDs exist in reference dataset")
        print("- Check for missing values in critical fields like user_id")
    
    print("\nProcessed New Users:")


C:\Users\Lenovo\AppData\Local\Temp\ipykernel_23532\775804979.py:645: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_encoded = pd.read_csv("Final_Anime_Dataset.csv")



Processed New Users:


c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\pipeline.py:62: FutureWarning: This Pipeline instance is not fitted yet. Call 'fit' with appropriate arguments before using other methods such as transform, predict, etc. This will raise an error in 1.8 instead of the current warning.
  warnings.warn(


In [6]:
final_user_features.columns

Index(['Mean Score', 'Completed', 'avg_Action', 'avg_Adventure',
       'avg_Avant Garde', 'avg_Award Winning', 'avg_Boys Love', 'avg_Comedy',
       'avg_Drama', 'avg_Ecchi', 'avg_Erotica', 'avg_Fantasy',
       'avg_Girls Love', 'avg_Gourmet', 'avg_Hentai', 'avg_Horror',
       'avg_Mystery', 'avg_Romance', 'avg_Sci-Fi', 'avg_Slice of Life',
       'avg_Sports', 'avg_Supernatural', 'avg_Suspense', 'Gender_Female',
       'Gender_Male', 'Gender_Non-Binary', 'Age', 'Age_Group__Gen_Alpha',
       'Age_Group__Zoomers', 'Age_Group__Millennials', 'Age_Group__Gen_X',
       'Age_Group__Boomers_Plus'],
      dtype='object')

In [7]:
X_U

array([[-0.7537492 ,  0.28942198, -0.40323402, ..., -1.44209169,
         1.4991455 , -0.10613655],
       [-0.00807431, -0.25010196, -0.04174073, ..., -1.44209169,
         1.4991455 , -0.10613655],
       [-0.58083908, -0.04050273, -0.60524498, ...,  0.69343719,
        -0.66704666, -0.10613655],
       ...,
       [ 0.14322205,  1.53537294,  0.213431  , ...,  0.69343719,
        -0.66704666, -0.10613655],
       [-0.48357714, -0.3160869 , -1.15811707, ...,  0.69343719,
        -0.66704666, -0.10613655],
       [-1.15360385,  0.08758568, -1.46644958, ...,  0.69343719,
        -0.66704666, -0.10613655]])

In [8]:
X_U.shape

(71278, 32)

In [10]:
user_data_temp_final.shape

(8467079, 10)

In [11]:
user_data_temp_final.columns

Index(['Unnamed: 0', 'Username', 'Gender', 'Mean Score', 'Completed',
       'Birth_Year', 'user_id', 'anime_id', 'Anime Title', 'rating'],
      dtype='object')

In [12]:
import pandas as pd
import numpy as np
import random
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from ipywidgets import VBox, HBox, Label, Button, Output, IntSlider, HTML as HTMLWidget

class AnimeRatingInterface:
    """
    Interactive interface for collecting user information, preferences, and anime ratings.
    Selects top anime by popularity within chosen genres and presents a random subset for rating.
    """
    
    def __init__(self, anime_data_path="Final_Anime_Dataset.csv"):
        """Initialize the interface with anime dataset."""
        self.anime_data = pd.read_csv(anime_data_path)
        self.user_info = None
        self.selected_genres = []
        self.ratings = []
        self.current_anime_idx = 0
        self.anime_to_rate = []
        self.output_data = None
        self.genre_columns = [
            'Genre_Action', 'Genre_Adventure', 'Genre_Avant Garde',
            'Genre_Award Winning', 'Genre_Boys Love', 'Genre_Comedy', 'Genre_Drama',
            'Genre_Ecchi', 'Genre_Erotica', 'Genre_Fantasy', 'Genre_Girls Love',
            'Genre_Gourmet', 'Genre_Hentai', 'Genre_Horror', 'Genre_Mystery',
            'Genre_Romance', 'Genre_Sci-Fi', 'Genre_Slice of Life', 'Genre_Sports',
            'Genre_Supernatural', 'Genre_Suspense'
        ]
        
        # Initialize the main notebook output
        self.main_output = Output()
        
        # Start the interface
        self.show_user_form()
        
    def show_user_form(self):
        """Display the initial user information form."""
        with self.main_output:
            clear_output(wait=True)
            print("=== Anime Rating Interface ===")
            print("\nPlease enter your information:")
        
        # User form elements
        self.username_input = widgets.Text(
            value='',
            placeholder='Enter your username',
            description='Username:',
            disabled=False
        )
        
        self.user_id_input = widgets.IntText(
            value=None,
            placeholder='Enter your user ID',
            description='User ID:',
            disabled=False
        )
        
        self.gender_dropdown = widgets.Dropdown(
            options=['Male', 'Female', 'Non-Binary'],
            value='Male',
            description='Gender:',
            disabled=False
        )
        
        current_year = 2025
        self.birth_year_slider = widgets.IntSlider(
            value=1995,
            min=1940,
            max=current_year-13,  # Assuming minimum age is 13
            step=1,
            description='Birth Year:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True
        )
        
        # Submit button
        submit_button = widgets.Button(
            description='Continue',
            button_style='primary',
            tooltip='Submit user information'
        )
        submit_button.on_click(self.on_user_form_submit)
        
        # Display the form
        form = VBox([
            self.username_input,
            self.user_id_input,
            self.gender_dropdown,
            self.birth_year_slider,
            submit_button
        ])
        
        display(self.main_output, form)
    
    def on_user_form_submit(self, button):
        """Handle user form submission and move to genre selection."""
        # Validate inputs
        if not self.username_input.value.strip():
            with self.main_output:
                clear_output(wait=True)
                print("Username cannot be empty. Please enter a username.")
            return
        
        # Store user information
        self.user_info = {
            'Username': self.username_input.value,
            'user_id': self.user_id_input.value,
            'Gender': self.gender_dropdown.value,
            'Birth_Year': self.birth_year_slider.value,
            'Mean Score': 0.0,  # Will be calculated after ratings
            'Completed': 0       # Will be incremented for each rated anime
        }
        
        # Move to genre selection
        self.show_genre_selection()
    
    def show_genre_selection(self):
        """Show genre selection checkboxes."""
        with self.main_output:
            clear_output(wait=True)
            print(f"Hello, {self.user_info['Username']}!")
            print("\nPlease select genres you're interested in:")
        
        # Create genre checkboxes
        genre_checkboxes = []
        for genre_col in self.genre_columns:
            genre_name = genre_col.replace('Genre_', '')
            checkbox = widgets.Checkbox(
                value=False,
                description=genre_name,
                disabled=False,
                indent=False
            )
            genre_checkboxes.append(checkbox)
        
        # Organize checkboxes in a grid layout (3 columns)
        rows = []
        for i in range(0, len(genre_checkboxes), 3):
            row = HBox(genre_checkboxes[i:i+3])
            rows.append(row)
        
        # Continue button
        continue_button = widgets.Button(
            description='Find Anime',
            button_style='primary',
            tooltip='Find anime based on selected genres'
        )
        continue_button.on_click(lambda b: self.on_genre_selection_submit(b, genre_checkboxes))
        
        # Display the form
        form = VBox(rows + [continue_button])
        
        display(self.main_output, form)
    
    def on_genre_selection_submit(self, button, genre_checkboxes):
        """Process selected genres and move to anime rating."""
        # Get selected genres
        self.selected_genres = []
        for i, checkbox in enumerate(genre_checkboxes):
            if checkbox.value:
                genre_col = self.genre_columns[i]
                self.selected_genres.append(genre_col)
        
        if not self.selected_genres:
            with self.main_output:
                clear_output(wait=True)
                print("Please select at least one genre.")
            return
        
        # Find anime that match the selected genres
        self.find_anime_to_rate()
        
        # Check if we found any anime
        if not self.anime_to_rate:
            with self.main_output:
                clear_output(wait=True)
                print("No anime found for the selected genres. Please try different genres.")
                
            # Return to genre selection
            self.show_genre_selection()
            return
        
        # Start rating process
        self.current_anime_idx = 0
        self.show_anime_rating()
    
    def find_anime_to_rate(self):
        """Find popular anime matching selected genres and select a random subset."""
        filtered_anime = self.anime_data.copy()
        
        # Filter anime with at least one selected genre
        filtered_anime = filtered_anime[filtered_anime[self.selected_genres].sum(axis=1) > 0]
        
        # Sort by popularity (using Members as indicator of popularity)
        if 'Members' in filtered_anime.columns:
            # Sort primarily by match score, then by popularity
            filtered_anime['match_score'] = filtered_anime[self.selected_genres].sum(axis=1)
            filtered_anime = filtered_anime.sort_values(['match_score', 'Members'], 
                                                       ascending=[False, False])
        else:
            # Fallback to Score if Members column doesn't exist
            filtered_anime['match_score'] = filtered_anime[self.selected_genres].sum(axis=1)
            filtered_anime = filtered_anime.sort_values(['match_score', 'Score'], 
                                                       ascending=[False, False])
        
        # Select top 40 popular anime matching the genres
        top_anime = filtered_anime.head(40)
        
        # Randomly select 20 from the top 40
        if len(top_anime) > 20:
            selected_indices = random.sample(range(len(top_anime)), 20)
            self.anime_to_rate = top_anime.iloc[selected_indices][
                ['anime_id', 'Name', 'Score', 'Genres', 'Synopsis']
            ].to_dict('records')
        else:
            self.anime_to_rate = top_anime[
                ['anime_id', 'Name', 'Score', 'Genres', 'Synopsis']
            ].to_dict('records')
        
        # Shuffle the final selection
        random.shuffle(self.anime_to_rate)
    
    def show_anime_rating(self):
        """Display the current anime for rating."""
        if self.current_anime_idx >= len(self.anime_to_rate):
            self.finalize_ratings()
            return
        
        current_anime = self.anime_to_rate[self.current_anime_idx]
        
        with self.main_output:
            clear_output(wait=True)
            print(f"Please rate these {len(self.anime_to_rate)} anime selected for you")
            print(f"Rating {self.current_anime_idx + 1} of {len(self.anime_to_rate)}")
        
        # Anime information
        anime_title = HTMLWidget(f"<h3>{current_anime['Name']}</h3>")
        score_info = widgets.HTML(f"<b>MAL Score:</b> {current_anime['Score']}")
        genres = widgets.HTML(f"<b>Genres:</b> {current_anime['Genres']}")
        
        # Truncate long synopsis
        synopsis = current_anime['Synopsis']
        if isinstance(synopsis, str) and len(synopsis) > 500:
            synopsis = synopsis[:500] + "..."
        elif not isinstance(synopsis, str):
            synopsis = "No synopsis available."
        synopsis_text = widgets.HTML(f"<b>Synopsis:</b><br>{synopsis}")
        
        # Rating slider
        rating_slider = widgets.IntSlider(
            value=7,
            min=1,
            max=10,
            step=1,
            description=f'Rating:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True
        )
        
        # Skip button and submit button side by side
        skip_button = widgets.Button(
            description='Skip',
            button_style='warning',
            tooltip='Skip rating this anime'
        )
        
        submit_button = widgets.Button(
            description='Submit Rating',
            button_style='success',
            tooltip='Submit your rating'
        )
        
        # Button handlers
        skip_button.on_click(lambda b: self.skip_rating())
        submit_button.on_click(lambda b: self.submit_rating(rating_slider.value))
        
        # Put buttons side by side
        buttons = HBox([skip_button, submit_button])
        
        # Display everything
        form = VBox([
            anime_title,
            score_info,
            genres,
            synopsis_text,
            rating_slider,
            buttons
        ])
        
        display(self.main_output, form)
    
    def skip_rating(self):
        """Skip rating the current anime."""
        self.current_anime_idx += 1
        self.show_anime_rating()
    
    def submit_rating(self, rating):
        """Submit a rating for the current anime."""
        current_anime = self.anime_to_rate[self.current_anime_idx]
        
        # Store the rating
        self.ratings.append({
            'anime_id': current_anime['anime_id'],
            'Anime Title': current_anime['Name'],
            'rating': rating
        })
        
        # Move to the next anime
        self.current_anime_idx += 1
        self.show_anime_rating()
    
    def finalize_ratings(self):
        """Create the final dataframe and display results."""
        if not self.ratings:
            with self.main_output:
                clear_output(wait=True)
                print("No ratings were provided. Please try again.")
            return
        
        # Update user info with completed count and mean score
        self.user_info['Completed'] = len(self.ratings)
        self.user_info['Mean Score'] = sum(r['rating'] for r in self.ratings) / len(self.ratings)
        
        # Create ratings dataframe
        ratings_df = pd.DataFrame(self.ratings)
        
        # Create user info dataframe (expanded to match the length of ratings)
        user_rows = []
        for _ in range(len(self.ratings)):
            user_rows.append(self.user_info.copy())
        user_df = pd.DataFrame(user_rows)
        
        # Combine into a single dataframe
        self.output_data = pd.concat([user_df, ratings_df], axis=1)
        
        # Reorder columns to match the expected format
        self.output_data = self.output_data[[
            'Username', 'Gender', 'Mean Score', 'Completed', 'Birth_Year',
            'user_id', 'anime_id', 'Anime Title', 'rating'
        ]]
        
        # Save to CSV
        filename = f"user_{self.user_info['user_id']}_ratings_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.csv"
        self.output_data.to_csv(filename, index=False)
        
        # Generate recommendations from the dataframe (optional)
        
        with self.main_output:
            clear_output(wait=True)
            print(f"Thank you for rating {len(self.ratings)} anime!")
            print(f"Your average rating: {self.user_info['Mean Score']:.2f}/10")
            print(f"\nRatings have been saved to: {filename}")
            
            # Show a preview of the ratings
            display(HTML("<h3>Your Ratings:</h3>"))
            display(self.output_data[['Anime Title', 'rating']].sort_values('rating', ascending=False))
            
            # Show a preview of the dataframe
            display(HTML("<h3>Complete Rating Data:</h3>"))
            display(self.output_data.head())
            print(f"DataFrame shape: {self.output_data.shape}")
            print("This data can now be used with our recommender system.")

# Run the interface
anime_interface = AnimeRatingInterface()

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_23532\737926107.py:16: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  self.anime_data = pd.read_csv(anime_data_path)


Output()

In [13]:
import pandas as pd

# Load preprocessed anime dataset (with features like genre, rating stats, etc.)
df_encoded = pd.read_csv("Final_Anime_Dataset.csv")

# Load new user ratings
new_user_raw = pd.read_csv("user_8974_ratings_20250414_224401.csv")

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_23532\4063214905.py:4: DtypeWarning: Columns (20) have mixed types. Specify dtype option on import or set low_memory=False.
  df_encoded = pd.read_csv("Final_Anime_Dataset.csv")


In [14]:
new_user_raw.head()
#'Username', 'Gender', 'Mean Score', 'Completed','Birth_Year', 'user_id', 'anime_id', 'Anime Title', 'rating'

,Username,Gender,Mean Score,Completed,Birth_Year,user_id,anime_id,Anime Title,rating
0,Kanishk Sharma,Male,7.2,20,1995,8974,2775,Genocyber,6
1,Kanishk Sharma,Male,7.2,20,1995,8974,22689,Terra Formars: Bugs 2-hen,7
2,Kanishk Sharma,Male,7.2,20,1995,8974,1346,AD Police,7
3,Kanishk Sharma,Male,7.2,20,1995,8974,22535,Kiseijuu: Sei no Kakuritsu,10
4,Kanishk Sharma,Male,7.2,20,1995,8974,999,Blue Seed 2,6


'Mean Score', 'Completed',
                'avg_Action', 'avg_Adventure', 'avg_Avant Garde', 'avg_Award Winning',
                'avg_Boys Love', 'avg_Comedy', 'avg_Drama', 'avg_Ecchi', 'avg_Erotica',
                'avg_Fantasy', 'avg_Girls Love', 'avg_Gourmet', 'avg_Hentai',
                'avg_Horror', 'avg_Mystery', 'avg_Romance', 'avg_Sci-Fi',
                'avg_Slice of Life', 'avg_Sports', 'avg_Supernatural', 'avg_Suspense',
                'Gender_Female', 'Gender_Male', 'Gender_Non-Binary','Age',
                 'Age_Group__Gen_Alpha', 'Age_Group__Zoomers',
                'Age_Group__Millennials', 'Age_Group__Gen_X', 'Age_Group__Boomers_Plus'

In [15]:
user_data = pd.read_csv("Final_User_Dataset.csv")
from sklearn.preprocessing import OneHotEncoder

# Initialize the OneHotEncoder
encoder = OneHotEncoder(sparse_output=False, drop=None, dtype=int)

# Fit and transform the 'Gender' column
gender_encoded = encoder.fit_transform(user_data[['Gender']])

# Get the column names for the encoded features
gender_columns = encoder.get_feature_names_out(['Gender'])

# Create a DataFrame for the encoded features
gender_encoded_df = pd.DataFrame(gender_encoded, columns=gender_columns)

# Concatenate the one-hot encoded columns with the original DataFrame
user_data = pd.concat([user_data, gender_encoded_df], axis=1)

# Drop the original 'Gender' column if no longer needed
user_data.drop(columns=['Gender'], inplace=True)

# Display the updated DataFrame
print(user_data.head())

user_input_relevant = user_data[['Mean Score', 'Completed',
                'avg_Action', 'avg_Adventure', 'avg_Avant Garde', 'avg_Award Winning',
                'avg_Boys Love', 'avg_Comedy', 'avg_Drama', 'avg_Ecchi', 'avg_Erotica',
                'avg_Fantasy', 'avg_Girls Love', 'avg_Gourmet', 'avg_Hentai',
                'avg_Horror', 'avg_Mystery', 'avg_Romance', 'avg_Sci-Fi',
                'avg_Slice of Life', 'avg_Sports', 'avg_Supernatural', 'avg_Suspense',
                'Gender_Female', 'Gender_Male', 'Gender_Non-Binary', 'Age',
                'Age_Group__Gen_Alpha', 'Age_Group__Zoomers',
                'Age_Group__Millennials', 'Age_Group__Gen_X', 'Age_Group__Boomers_Plus']]

   user_id  Username  Mean Score  Completed  avg_Action  avg_Adventure  \
0        1     Xinil        7.37      233.0        7.68           7.90   
1       20    vondur        8.06       94.0        8.02           8.24   
2       66    Hiromi        7.53      148.0        7.49           7.31   
3       82    Achtor        7.17      153.0        7.36           7.71   
4      112  luffykun        8.77      125.0        8.60           8.36   

   avg_Avant Garde  avg_Award Winning  avg_Boys Love  avg_Comedy  ...  \
0             8.00               8.18           7.06        7.02  ...   
1             9.00               8.43           7.06        7.77  ...   
2             7.00               7.90           7.06        7.46  ...   
3             8.80               8.25           7.06        7.05  ...   
4             7.93               9.80           7.06        8.67  ...   

   Age_Group__Gen_X  Age_Group__Boomers_Plus  Category_Classic_Era_Fans  \
0                 1                      

In [16]:
user_input_relevant.head()

,Mean Score,Completed,avg_Action,avg_Adventure,avg_Avant Garde,avg_Award Winning,avg_Boys Love,avg_Comedy,avg_Drama,avg_Ecchi,...,avg_Suspense,Gender_Female,Gender_Male,Gender_Non-Binary,Age,Age_Group__Gen_Alpha,Age_Group__Zoomers,Age_Group__Millennials,Age_Group__Gen_X,Age_Group__Boomers_Plus
0,7.37,233.0,7.68,7.90,8.00,8.18,7.06,7.02,7.67,6.50,...,8.18,0,1,0,40,0,0,0,1,0
1,8.06,94.0,8.02,8.24,9.00,8.43,7.06,7.77,8.04,5.67,...,8.56,0,1,0,37,0,0,0,1,0
2,7.53,148.0,7.49,7.31,7.00,7.90,7.06,7.46,7.60,7.43,...,7.00,0,1,0,35,0,0,1,0,0
3,7.17,153.0,7.36,7.71,8.80,8.25,7.06,7.05,7.44,6.06,...,8.18,0,1,0,36,0,0,0,1,0
4,8.77,125.0,8.60,8.36,7.93,9.80,7.06,8.67,9.04,8.33,...,10.00,0,1,0,42,0,0,0,1,0


In [28]:
class MissingValueImputer(BaseEstimator, TransformerMixin):
    """
    A transformer that imputes missing values in the user feature matrix.
    
    Parameters:
    -----------
    strategy : str, default='mean'
        Imputation strategy ('mean', 'median', 'most_frequent', 'constant')
    fill_value : any, default=None
        Used when strategy is 'constant'
    """
    
    def __init__(self, strategy='mean', fill_value=None):
        self.strategy = strategy
        self.fill_value = fill_value
        self.imputer = SimpleImputer(strategy=strategy, fill_value=fill_value)
        self.feature_names_in_ = None
    
    def fit(self, X, y=None):
        """
        Fit the imputer to the data.
        
        Parameters:
        -----------
        X : pandas.DataFrame
            Input DataFrame
        y : array-like, default=None
            Not used, present for API consistency
            
        Returns:
        --------
        self
        """
        self.feature_names_in_ = X.columns
        self.imputer.fit(X)
        return self
    
    def transform(self, X):
        """
        Impute missing values in the data.
        
        Parameters:
        -----------
        X : pandas.DataFrame
            Input DataFrame
            
        Returns:
        --------
        pandas.DataFrame
            DataFrame with imputed values
        """
        imputed_data = self.imputer.transform(X)
        return pd.DataFrame(imputed_data, columns=self.feature_names_in_, index=X.index)
    
    def get_feature_names_out(self, input_features=None):
        """Get output feature names."""
        return list(self.feature_names_in_)

In [17]:
missvalueimpute=MissingValueImputer()
missvalueimpute.fit(user_input_relevant)


MissingValueImputer()

In [19]:
df_encoded.columns

Index(['anime_id', 'Name', 'English name', 'Other name', 'Score', 'Genres',
       'Synopsis', 'Episodes', 'Aired', 'Premiered',
       ...
       'Studio_pH Studio, D & D Pictures', 'Studio_pH Studio, Noovo',
       'Studio_production doA', 'Studio_studio MOTHER', 'Studio_studio YOG',
       'Studio_trenova', 'Studio_ufotable',
       'Studio_ufotable, Shaft, A-1 Pictures, SILVER LINK., Lerche, Lay-duce, CloverWorks, Drive',
       'Studio_ufotable, feel., Studio Flag', 'Release_Year'],
      dtype='object', length=1571)

In [20]:
user_input_df.shape

(20, 9)

In [18]:


# Select necessary columns
user_input_df = new_user_raw

# Create a pipeline that merges and processes the data
full_pipeline = create_user_preprocessing_pipeline(df_encoded, user_input_df)

# Fit-transform to get the processed features for the new user
processed_user_features = full_pipeline.fit_transform(user_input_df)
processed_user_features= missvalueimpute.transform(processed_user_features)

# Get the final matrix (as numpy array)
X_U = processed_user_features.values

print(f"X_U matrix shape: {X_U.shape}")


c:\Users\Lenovo\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\impute\_base.py:635: UserWarning: Skipping features without any observed values: ['avg_Avant Garde' 'avg_Award Winning' 'avg_Boys Love' 'avg_Erotica'
 'avg_Gourmet' 'avg_Hentai' 'avg_Slice of Life' 'avg_Sports']. At least one non-missing value is needed for imputation with strategy='mean'.
  warnings.warn(


ValueError: Shape of passed values is (1, 22), indices imply (1, 30)

In [ ]:
X_U

array([[-0.7537492 ,  0.28942198, -0.40323402, ..., -1.44209169,
         1.4991455 , -0.10613655],
       [-0.00807431, -0.25010196, -0.04174073, ..., -1.44209169,
         1.4991455 , -0.10613655],
       [-0.58083908, -0.04050273, -0.60524498, ...,  0.69343719,
        -0.66704666, -0.10613655],
       ...,
       [ 0.14322205,  1.53537294,  0.213431  , ...,  0.69343719,
        -0.66704666, -0.10613655],
       [-0.48357714, -0.3160869 , -1.15811707, ...,  0.69343719,
        -0.66704666, -0.10613655],
       [-1.15360385,  0.08758568, -1.46644958, ...,  0.69343719,
        -0.66704666, -0.10613655]])

In [ ]:
import pandas as pd
import numpy as np
import random
from IPython.display import display, HTML, clear_output
import ipywidgets as widgets
from ipywidgets import VBox, HBox, Label, Button, Output, IntSlider, HTML as HTMLWidget

class AnimeRatingInterface:
    """
    Interactive interface for collecting user information, preferences, and anime ratings.
    Selects top anime by popularity within chosen genres and presents a random subset for rating.
    """
    
    def __init__(self, anime_data_path="Final_Anime_Dataset.csv"):
        """Initialize the interface with anime dataset."""
        self.anime_data = pd.read_csv(anime_data_path)
        self.user_info = None
        self.selected_genres = []
        self.ratings = []
        self.current_anime_idx = 0
        self.anime_to_rate = []
        self.output_data = None
        self.genre_columns = [
            'Genre_Action', 'Genre_Adventure', 'Genre_Avant Garde',
            'Genre_Award Winning', 'Genre_Boys Love', 'Genre_Comedy', 'Genre_Drama',
            'Genre_Ecchi', 'Genre_Erotica', 'Genre_Fantasy', 'Genre_Girls Love',
            'Genre_Gourmet', 'Genre_Hentai', 'Genre_Horror', 'Genre_Mystery',
            'Genre_Romance', 'Genre_Sci-Fi', 'Genre_Slice of Life', 'Genre_Sports',
            'Genre_Supernatural', 'Genre_Suspense'
        ]
        
        # Initialize the main notebook output
        self.main_output = Output()
        
        # Start the interface
        self.show_user_form()
        
    def show_user_form(self):
        """Display the initial user information form."""
        with self.main_output:
            clear_output(wait=True)
            print("=== Anime Rating Interface ===")
            print("\nPlease enter your information:")
        
        # User form elements
        self.username_input = widgets.Text(
            value='',
            placeholder='Enter your username',
            description='Username:',
            disabled=False
        )
        
        self.user_id_input = widgets.IntText(
            value=None,
            placeholder='Enter your user ID',
            description='User ID:',
            disabled=False
        )
        
        self.gender_dropdown = widgets.Dropdown(
            options=['Male', 'Female', 'Non-Binary'],
            value='Male',
            description='Gender:',
            disabled=False
        )
        
        current_year = 2025
        self.birth_year_slider = widgets.IntSlider(
            value=1995,
            min=1940,
            max=current_year-13,  # Assuming minimum age is 13
            step=1,
            description='Birth Year:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True
        )
        
        # Submit button
        submit_button = widgets.Button(
            description='Continue',
            button_style='primary',
            tooltip='Submit user information'
        )
        submit_button.on_click(self.on_user_form_submit)
        
        # Display the form
        form = VBox([
            self.username_input,
            self.user_id_input,
            self.gender_dropdown,
            self.birth_year_slider,
            submit_button
        ])
        
        display(self.main_output, form)
    
    def on_user_form_submit(self, button):
        """Handle user form submission and move to genre selection."""
        # Validate inputs
        if not self.username_input.value.strip():
            with self.main_output:
                clear_output(wait=True)
                print("Username cannot be empty. Please enter a username.")
            return
        
        # Store user information
        self.user_info = {
            'Username': self.username_input.value,
            'user_id': self.user_id_input.value,
            'Gender': self.gender_dropdown.value,
            'Birth_Year': self.birth_year_slider.value,
            'Mean Score': 0.0,  # Will be calculated after ratings
            'Completed': 0       # Will be incremented for each rated anime
        }
        
        # Move to genre selection
        self.show_genre_selection()
    
    def show_genre_selection(self):
        """Show genre selection checkboxes."""
        with self.main_output:
            clear_output(wait=True)
            print(f"Hello, {self.user_info['Username']}!")
            print("\nPlease select genres you're interested in:")
        
        # Create genre checkboxes
        genre_checkboxes = []
        for genre_col in self.genre_columns:
            genre_name = genre_col.replace('Genre_', '')
            checkbox = widgets.Checkbox(
                value=False,
                description=genre_name,
                disabled=False,
                indent=False
            )
            genre_checkboxes.append(checkbox)
        
        # Organize checkboxes in a grid layout (3 columns)
        rows = []
        for i in range(0, len(genre_checkboxes), 3):
            row = HBox(genre_checkboxes[i:i+3])
            rows.append(row)
        
        # Continue button
        continue_button = widgets.Button(
            description='Find Anime',
            button_style='primary',
            tooltip='Find anime based on selected genres'
        )
        continue_button.on_click(lambda b: self.on_genre_selection_submit(b, genre_checkboxes))
        
        # Display the form
        form = VBox(rows + [continue_button])
        
        display(self.main_output, form)
    
    def on_genre_selection_submit(self, button, genre_checkboxes):
        """Process selected genres and move to anime rating."""
        # Get selected genres
        self.selected_genres = []
        for i, checkbox in enumerate(genre_checkboxes):
            if checkbox.value:
                genre_col = self.genre_columns[i]
                self.selected_genres.append(genre_col)
        
        if not self.selected_genres:
            with self.main_output:
                clear_output(wait=True)
                print("Please select at least one genre.")
            return
        
        # Find anime that match the selected genres
        self.find_anime_to_rate()
        
        # Check if we found any anime
        if not self.anime_to_rate:
            with self.main_output:
                clear_output(wait=True)
                print("No anime found for the selected genres. Please try different genres.")
                
            # Return to genre selection
            self.show_genre_selection()
            return
        
        # Start rating process
        self.current_anime_idx = 0
        self.show_anime_rating()
    
    def find_anime_to_rate(self):
        """Find popular anime matching selected genres and select a random subset."""
        filtered_anime = self.anime_data.copy()
        
        # Filter anime with at least one selected genre
        filtered_anime = filtered_anime[filtered_anime[self.selected_genres].sum(axis=1) > 0]
        
        # Sort by popularity (using Members as indicator of popularity)
        if 'Members' in filtered_anime.columns:
            # Sort primarily by match score, then by popularity
            filtered_anime['match_score'] = filtered_anime[self.selected_genres].sum(axis=1)
            filtered_anime = filtered_anime.sort_values(['match_score', 'Members'], 
                                                       ascending=[False, False])
        else:
            # Fallback to Score if Members column doesn't exist
            filtered_anime['match_score'] = filtered_anime[self.selected_genres].sum(axis=1)
            filtered_anime = filtered_anime.sort_values(['match_score', 'Score'], 
                                                       ascending=[False, False])
        
        # Select top 40 popular anime matching the genres
        top_anime = filtered_anime.head(40)
        
        # Randomly select 20 from the top 40
        if len(top_anime) > 20:
            selected_indices = random.sample(range(len(top_anime)), 20)
            self.anime_to_rate = top_anime.iloc[selected_indices][
                ['anime_id', 'Name', 'Score', 'Genres', 'Synopsis']
            ].to_dict('records')
        else:
            self.anime_to_rate = top_anime[
                ['anime_id', 'Name', 'Score', 'Genres', 'Synopsis']
            ].to_dict('records')
        
        # Shuffle the final selection
        random.shuffle(self.anime_to_rate)
    
    def show_anime_rating(self):
        """Display the current anime for rating."""
        if self.current_anime_idx >= len(self.anime_to_rate):
            self.finalize_ratings()
            return
        
        current_anime = self.anime_to_rate[self.current_anime_idx]
        
        with self.main_output:
            clear_output(wait=True)
            print(f"Please rate these {len(self.anime_to_rate)} anime selected for you")
            print(f"Rating {self.current_anime_idx + 1} of {len(self.anime_to_rate)}")
        
        # Anime information
        anime_title = HTMLWidget(f"<h3>{current_anime['Name']}</h3>")
        score_info = widgets.HTML(f"<b>MAL Score:</b> {current_anime['Score']}")
        genres = widgets.HTML(f"<b>Genres:</b> {current_anime['Genres']}")
        
        # Truncate long synopsis
        synopsis = current_anime['Synopsis']
        if isinstance(synopsis, str) and len(synopsis) > 500:
            synopsis = synopsis[:500] + "..."
        elif not isinstance(synopsis, str):
            synopsis = "No synopsis available."
        synopsis_text = widgets.HTML(f"<b>Synopsis:</b><br>{synopsis}")
        
        # Rating slider
        rating_slider = widgets.IntSlider(
            value=7,
            min=1,
            max=10,
            step=1,
            description=f'Rating:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True
        )
        
        # Skip button and submit button side by side
        skip_button = widgets.Button(
            description='Skip',
            button_style='warning',
            tooltip='Skip rating this anime'
        )
        
        submit_button = widgets.Button(
            description='Submit Rating',
            button_style='success',
            tooltip='Submit your rating'
        )
        
        # Button handlers
        skip_button.on_click(lambda b: self.skip_rating())
        submit_button.on_click(lambda b: self.submit_rating(rating_slider.value))
        
        # Put buttons side by side
        buttons = HBox([skip_button, submit_button])
        
        # Display everything
        form = VBox([
            anime_title,
            score_info,
            genres,
            synopsis_text,
            rating_slider,
            buttons
        ])
        
        display(self.main_output, form)
    
    def skip_rating(self):
        """Skip rating the current anime."""
        self.current_anime_idx += 1
        self.show_anime_rating()
    
    def submit_rating(self, rating):
        """Submit a rating for the current anime."""
        current_anime = self.anime_to_rate[self.current_anime_idx]
        
        # Store the rating
        self.ratings.append({
            'anime_id': current_anime['anime_id'],
            'Anime Title': current_anime['Name'],
            'rating': rating
        })
        
        # Move to the next anime
        self.current_anime_idx += 1
        self.show_anime_rating()
    
    def finalize_ratings(self):
        """Create the final dataframe and display results."""
        if not self.ratings:
            with self.main_output:
                clear_output(wait=True)
                print("No ratings were provided. Please try again.")
            return
        
        # Update user info with completed count and mean score
        self.user_info['Completed'] = len(self.ratings)
        self.user_info['Mean Score'] = sum(r['rating'] for r in self.ratings) / len(self.ratings)
        
        # Create ratings dataframe
        ratings_df = pd.DataFrame(self.ratings)
        
        # Create user info dataframe (expanded to match the length of ratings)
        user_rows = []
        for _ in range(len(self.ratings)):
            user_rows.append(self.user_info.copy())
        user_df = pd.DataFrame(user_rows)
        
        # Combine into a single dataframe
        self.output_data = pd.concat([user_df, ratings_df], axis=1)
        
        # Reorder columns to match the expected format
        self.output_data = self.output_data[[
            'Username', 'Gender', 'Mean Score', 'Completed', 'Birth_Year',
            'user_id', 'anime_id', 'Anime Title', 'rating'
        ]]
        
        # Save to CSV
        filename = f"user_{self.user_info['user_id']}_ratings_{pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')}.csv"
        self.output_data.to_csv(filename, index=False)
        
        # Generate recommendations from the dataframe (optional)
        
        with self.main_output:
            clear_output(wait=True)
            print(f"Thank you for rating {len(self.ratings)} anime!")
            print(f"Your average rating: {self.user_info['Mean Score']:.2f}/10")
            print(f"\nRatings have been saved to: {filename}")
            
            # Show a preview of the ratings
            display(HTML("<h3>Your Ratings:</h3>"))
            display(self.output_data[['Anime Title', 'rating']].sort_values('rating', ascending=False))
            
            # Show a preview of the dataframe
            display(HTML("<h3>Complete Rating Data:</h3>"))
            display(self.output_data.head())
            print(f"DataFrame shape: {self.output_data.shape}")
            print("This data can now be used with our recommender system.")

# Run the interface
anime_interface = AnimeRatingInterface()
# Load anime metadata (preprocessed)
df_encoded = pd.read_csv("Final_Anime_Dataset.csv")

# Load new user data from interface
new_user_raw = pd.read_csv("user_1975_ratings_20250414_203122.csv")  # From AnimeRatingInterface
user_input_df = new_user_raw[["user_id", "anime_id", "rating"]]

df_encoded = pd.read_csv('Final_Anime_Dataset.csv')

        
# Create and apply pipeline for raw data
full_pipeline = create_user_preprocessing_pipeline(df_encoded, user_input_df)
processed_user_features = full_pipeline.fit_transform(user_data_temp_final)
print(f"Processed user features shape: {processed_user_features.shape}")
        
# Alternatively, if user data is already processed
user_data = pd.read_csv('Final_User_Dataset.csv')
partial_pipeline = create_user_preprocessing_pipeline(df_encoded)
final_user_features = partial_pipeline.fit_transform(user_data)
print(f"Final user features shape: {final_user_features.shape}")
        
# Convert to numpy array for model input if needed
X_U = final_user_features.values
print(f"X_U matrix shape: {X_U.shape}")


import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split


model=tf.keras.models.load_model('../saved_model/recommender_model_6.h5')

# Utilize the model to predict ratings for the user from the anime_dataset 



